In [1]:
from src.config import filepaths
import numpy as np
import matplotlib.pyplot as plt

In [2]:
smfp = filepaths['artifacts_sim_mats']

In [3]:
dataset = 'sprhea'
toc = 'v3_folded_pt_ns'

rxn_sim_metrics = [
    'bag_of_molecules_reactions',
    'rc_agg_r2_reactions',
    'rc_agg_r3_reactions',
    'rc_agg_r4_reactions',
    "rcmcs"
]

rxn_sim_mats = {name: np.load(smfp / f"{dataset}_{toc}_{name}.npy")
                for name in rxn_sim_metrics}



In [4]:
n = rxn_sim_mats['rcmcs'].shape[0]
idxs = []
for i in range(n - 1):
    for j in range(i + 1, n):
        idxs.append((i, j))

i, j = [np.array(elt) for elt in zip(*idxs)]

: 

In [5]:
sim_mats = ['bag_of_molecules_reactions', 'rcmcs']
data = []
for sm in sim_mats:
    data.append(rxn_sim_mats[sm][i, j])

x, y = data
fig, ax = plt.subplots()
ax.plot(x, y, marker='.', color='black')
plt.show()

: 

: 